## Using neural networks to predict on Kryptonite-9 dataset

In [1]:
import numpy as np
import os
import random
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from itertools import product

In [2]:
print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Device: {0}'.format(device))

myseed = 6095 

random.seed(myseed)
os.environ['PYTHONHASHSEED'] = str(myseed)
np.random.seed(myseed)
torch.manual_seed(myseed)
torch.cuda.manual_seed(myseed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True


2.5.1+cu118
Device: cuda


In [3]:
class NeuralNetwork(nn.Module):
    def __init__(self, input_size=9, hidden_size=5):
        super().__init__()
        self.linear_layer_stack = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size,hidden_size), 
            nn.ReLU(),
            nn.Linear(hidden_size,1), 
        ) 
        self._initialize_weights()  

    def _initialize_weights(self):
        for layer in self.linear_layer_stack:
            if isinstance(layer, nn.Linear):
                torch.nn.init.xavier_uniform_(layer.weight)
                torch.nn.init.zeros_(layer.bias)


    def forward(self, x):
        return self.linear_layer_stack(x)

In [4]:



# train_size = len(X_train)

def train_nn(model, dataloader, criterion, optimizer, epochs=10):
    for epoch in range(epochs):
        model.train()
        epoch_loss = 0
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.unsqueeze(1).to(device)  # Move data to device
            
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            epoch_loss += loss.item()
        
        print(f'Epoch {epoch+1}/{epochs}, Loss: {epoch_loss/len(dataloader):.4f}')



#### Validation

In [5]:
def validate_nn(model, X_val, y_val):
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val.to(device))
        val_outputs = torch.round(torch.sigmoid(val_outputs)).cpu().numpy()
        accuracy = accuracy_score(y_val, val_outputs)
    return accuracy

#### Grid Search Hyperparameter Tuning

In [6]:
def grid_search(X_train, y_train, X_val, y_val, param_grid, krypto_n):
    best_accuracy = 0
    best_params = None
    best_model = None
    
    for hidden_size, learning_rate, batch_size, epochs in product(*param_grid.values()):
        print(f"Training with hidden_size={hidden_size}, learning_rate={learning_rate}, batch_size={batch_size}, epochs={epochs}")
        
        model = NeuralNetwork(input_size=krypto_n, hidden_size=hidden_size).to(device)
        criterion = nn.BCEWithLogitsLoss()
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        
        train_dataset = TensorDataset(X_train, y_train)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        train_nn(model, train_loader, criterion, optimizer, epochs=epochs)
        
        accuracy = validate_nn(model, X_val, y_val)
        print(f"Validation Accuracy: {accuracy:.4f}")
        
        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_params = {'hidden_size': hidden_size, 'learning_rate': learning_rate, 
                           'batch_size': batch_size, 'epochs': epochs}
            best_model = model

        print(f"Current best accuracy: {best_accuracy:.4f}")

    
    print("Best Parameters:", best_params)
    print("Best Validation Accuracy:", best_accuracy)
    print(f"Krypto variant: {krypto_n}")
    return best_model, best_params

In [7]:
print(torch.__version__)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = "cpu"
print('Device: {0}'.format(device))


n = 9
X = np.load('Datasets/kryptonite-%s-X.npy'%(n))
y = np.load('Datasets/kryptonite-%s-y.npy'%(n))

# Shuffle and split the data
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.6, random_state=myseed)  # 60% training
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=myseed)  # 20% validation, 20% test

X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.float32)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.float32)



# param_grid = {
#     'hidden_size': [4, 8, 16], 
#     'learning_rate': [0.001, 0.01 , 0.1], 
#     'batch_size': [16, 32, 64],     
#     'epochs': [5, 10, 20,]              
# }


param_grid = {
    'hidden_size': [20],
    'learning_rate': [0.007],
    'batch_size': [64],     
    'epochs': [5]               
}


best_model, best_params = grid_search(X_train, y_train, X_val, y_val, param_grid, n)

# Current Hyperparameters for each Kryptonite Variant:
# Best Parameters: {'hidden_size': 16, 'learning_rate': 0.01, 'batch_size': 64, 'epochs': 20}
# Best Validation Accuracy: 0.9084722222222222 (subject to slight fluctuations due to non-deterministic nature of parallel computation used in training)
# Krypto variant: 12

# Best Parameters: {'hidden_size': 20, 'learning_rate': 0.007, 'batch_size': 64, 'epochs': 5}
# Best Validation Accuracy: 0.9538888888888889
# Krypto variant: 9



2.5.1+cu118
Device: cuda
Training with hidden_size=20, learning_rate=0.007, batch_size=64, epochs=5
Epoch 1/5, Loss: 0.6945
Epoch 2/5, Loss: 0.6648
Epoch 3/5, Loss: 0.4941
Epoch 4/5, Loss: 0.3433
Epoch 5/5, Loss: 0.2946
Validation Accuracy: 0.9156
Current best accuracy: 0.9156
Best Parameters: {'hidden_size': 20, 'learning_rate': 0.007, 'batch_size': 64, 'epochs': 5}
Best Validation Accuracy: 0.9155555555555556
Krypto variant: 9
